In [2]:
import json
from os.path import join
import jsonlines
def parse(self, response):
    jsonresponse=json.loads(response.text)
    with jsonlines.open('output.jsonl',mode='a') as writer:
        writer.write(jsonresponse)

def get_dataset(file_path):
    dataset = []
    with open(file_path, "r+", encoding="utf8") as f:
        for item in jsonlines.Reader(f):
            dataset.append(item)
    return dataset

train_data = get_dataset(join('data','train.jsonl'))
val_data = get_dataset(join('data','valid.jsonl'))
test_data = get_dataset(join('data','test_public.jsonl'))

In [3]:
import numpy as np

def argmax(scores):
    max_score = max(scores)
    return [i for i, s in enumerate(scores) if s == max_score]
def argmin(scores):
    min_id = []
    min_score = min(scores)
    for i, s in enumerate(scores):
        if s == min_score:
            min_id.append(i)
    return min_id
def choice_cooccurrence(choices):
    num = len(choices)
    choices_return = []
    scores = [0] * num
    choice_tokens = [[] for _ in range(num)]
    for i, choice in enumerate(choices):
        choices_return.append(choice)
        for c in choice:
            if c in choice_tokens[i]:
                #assert 1
                continue
            choice_tokens[i].append(c)
    for i in range(num):
        for j in range(num):
            if i == j:
                continue
            for c in choice_tokens[i]:
                if c in choice_tokens[j]:
                    scores[i] += 1
    return np.array(scores)
    
def translation_cooccurrence(translation, choices):
    scores = [0] * len(choices)
    for i, choice in enumerate(choices):
        choice_tokens = []
        for c in choice:
            if c in choice_tokens:
                continue
                # assert 1
            if c in translation:
                scores[i] += 1
            choice_tokens.append(c)
    return np.array(scores)

def position_cooccurrence(translation, choices):
    scores = [0] * len(choices)
    for i, choice in enumerate(choices):
        token_pos = []
        for c in choice:
            if c in translation:
                token_pos.append(translation.index(c))
        for j in range(len(token_pos)):
            for k in range(j):
                if token_pos[k] > token_pos[j]:
                    scores[i] += 1
        if len(token_pos) > 0:
            scores[i] /= len(token_pos)
    ids = argmin(scores)
    if len(ids) == 0:
        a =1
    return ids

In [4]:
wrong_list = [[] for _ in range(5)]
muti_list = []
nums = [0 for _ in range(5)]
acc_nums = [0 for _ in range(5)]
acc_num = 0
num = 0

for e in val_data:
    scores_choice_co = choice_cooccurrence(e['choices'])
    scores_translation_co = translation_cooccurrence(e['translation'], e['choices'])

    scores = scores_choice_co+scores_translation_co
    max_score = max(scores)
    max_idx = [idx for idx, s in enumerate(scores) if s == max_score]
    if scores[e['answer']] == max_score:
        acc_nums[len(max_idx)] += 1
    else:
        wrong_list[len(max_idx)].append((e,max_idx))
    nums[len(max_idx)] += 1
#print(acc_num/len(val_data))
for i in range(5):
    print(f'{i}:{acc_nums[i]}/{nums[i]}')
    if nums[i] == 0:
        print(f'{i}:{0}/{0}')
    else:
        print(acc_nums[i] / nums[i])
print(sum(acc_nums)/sum(nums))


0:0/0
0:0/0
1:2249/2347
0.9582445675330209
2:251/273
0.9194139194139194
3:66/70
0.9428571428571428
4:30/30
1.0
0.9544117647058824


In [5]:
with open('CCPM.jsonl','w',encoding = 'utf-8') as jfile:
    for test_data in test_data_new:
        jfile.write(json.dumps(test_data, ensure_ascii=False) + '\n')

NameError: name 'test_data_new' is not defined